In [112]:
import requests
from datetime import datetime
from elasticsearch import Elasticsearch
import nltk
import json
import pandas as pd
from elasticsearch.exceptions import NotFoundError

https://towardsdatascience.com/getting-started-with-elasticsearch-in-python-c3598e718380

# Basic Operations

Using the Elasticsearch package

In [4]:
def connect_to_elasticsearch():
    _es = None
    _es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
    if _es.ping():
        print('Connection Established')
    else:
        print('Something went wrong with the connection!')
    return _es

## Sanity Checks

In [27]:
# es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es = connect_to_elasticsearch()

Connection Established


In [44]:
# Get all indices within the ElasticSearch Instance.
indices = es.indices.get_mapping().keys()
real_index = []
for ind in indices:
    if not ind.startswith('.'):
        real_index.append(ind)
        
print(real_index)

['products']


In [53]:
def get_index_mapping(index_name):
    return es.indices.get_mapping()[index_name]

mapping = get_index_mapping('products')
# print(json.dumps(mapping, sort_keys=True,indent=4,))

In [58]:
mapping_df= pd.DataFrame.from_dict(mapping['mappings']['properties'], orient='index')

,type,fields,format
category,text,"{'keyword': {'type': 'keyword', 'ignore_above'...",NaN
created,date,NaN,yyyy/MM/dd HH:mm:ss||yyyy/MM/dd||epoch_millis
description,text,"{'keyword': {'type': 'keyword', 'ignore_above'...",NaN
in_stock,long,NaN,NaN
is_active,boolean,NaN,NaN
name,text,"{'keyword': {'type': 'keyword', 'ignore_above'...",NaN
price,long,NaN,NaN
sold,long,NaN,NaN
tags,text,"{'keyword': {'type': 'keyword', 'ignore_above'...",NaN


## Creating Index

Using the requests package

In [60]:
def create_index_1(index_name):
    es_path = "http://localhost:9200/"
    output = requests.put(es_path + index_name)
    return output

In [71]:
def create_index_2(es_instance, index_name):
    res = es_instance.indices.create(index=index_name)
    return res

In [64]:
res = create_index_1("world_places")

In [78]:
res = create_index_2(es, "activities")

In [79]:
res

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'activities'}

## Deleting index

In [92]:
class NotFoundError(Exception):
    pass

In [96]:
NotFoundError()

__main__.NotFoundError()

In [84]:
def delete_index_1(index_name):
    es_path = "http://localhost:9200/"
    output = requests.delete(es_path + index_name)
    return output.content

In [119]:
def delete_index_2(es_instance, index_name):
    try:
        res = es.indices.delete(index=index_name)
    except NotFoundError:
        res = 'Index -{}- does not exist'.format(index_name)
    return res

In [98]:
res = delete_index_1("activities")

In [120]:
res = delete_index_2(es, "activities")

In [121]:
res

'Index -activities- does not exist'

## Adding documents

Create a json-like object as a document to inject within the index

In [8]:
doc = {
    'author': 'kimchy',
    'text': 'Elasticsearch: cool. bonsai cool.',
    'timestamp': datetime.now(),
}

In [9]:
def push_doc_es(es_instance, index_name, doc_id, doc):
    res = es_instance.index(index=index_name, id=doc_id, body=doc)
    return res['result']

In [10]:
res = es.get(index="products", id=1)
print(res)
# print(res['_source'])
print(res.keys())
print(res['_source'].keys())

{'_index': 'products', '_type': '_doc', '_id': '1', '_version': 1, '_seq_no': 0, '_primary_term': 1, 'found': True, '_source': {'name': 'Wine - Maipo Valle Cabernet', 'price': 152, 'in_stock': 38, 'sold': 47, 'tags': ['Alcohol', 'Wine'], 'description': 'Aliquam augue quam, sollicitudin vitae, consectetuer eget, rutrum at, lorem. Integer tincidunt ante vel ipsum. Praesent blandit lacinia erat. Vestibulum sed magna at nunc commodo placerat. Praesent blandit. Nam nulla. Integer pede justo, lacinia eget, tincidunt eget, tempus vel, pede. Morbi porttitor lorem id ligula.', 'is_active': True, 'created': '2004/05/13'}}
dict_keys(['_index', '_type', '_id', '_version', '_seq_no', '_primary_term', 'found', '_source'])
dict_keys(['name', 'price', 'in_stock', 'sold', 'tags', 'description', 'is_active', 'created'])


In [12]:
res = es.search(index="products", body={"query": {"match_all": {}}})
print(len(res['hits']['hits']))
for prod in res['hits']['hits']:
    print(prod['_source']['name'])
    print(prod['_source']['in_stock'])    

10
Wine - Maipo Valle Cabernet
38
Tart Shells - Savory
10
Kirsch - Schloss
24
Coffee - Colombian Portioned
37
Venison - Liver
24
Nestea - Iced Tea
23
Beets - Pickled
25
Energy Drink Red Bull
25
Onions - Green
22
Ecolab - Hobart Washarm End Cap
25


In [13]:
res = es.update_by_query(index="products", 
                         body={"query": {"match_all": {}}, 
                               "script" : {"source": "ctx._source.in_stock--"}})

In [22]:
res = es.search(index="products", body={"query": {"match_all": {}}})
print(len(res['hits']['hits']))
for prod in res['hits']['hits']:
    print(prod['_id'])
    print(prod['_source']['name'])
    print(prod['_source']['in_stock'])

10
2
Tart Shells - Savory
9
3
Kirsch - Schloss
23
4
Coffee - Colombian Portioned
36
5
Venison - Liver
23
6
Nestea - Iced Tea
22
7
Beets - Pickled
24
8
Energy Drink Red Bull
24
9
Onions - Green
21
10
Ecolab - Hobart Washarm End Cap
24
11
Tuna - Salad Premix
3


In [19]:
es.update(index="products", id=1, 
          body=
          {"doc":{
              "category" : ["food"]
              }
          })

{'_index': 'products',
 '_type': '_doc',
 '_id': '1',
 '_version': 3,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 2000,
 '_primary_term': 1}

In [24]:
res = es.get(index="products", id=1)

In [25]:
res

{'_index': 'products',
 '_type': '_doc',
 '_id': '1',
 '_version': 3,
 '_seq_no': 2000,
 '_primary_term': 1,
 'found': True,
 '_source': {'sold': 47,
  'is_active': True,
  'price': 152,
  'created': '2004/05/13',
  'name': 'Wine - Maipo Valle Cabernet',
  'description': 'Aliquam augue quam, sollicitudin vitae, consectetuer eget, rutrum at, lorem. Integer tincidunt ante vel ipsum. Praesent blandit lacinia erat. Vestibulum sed magna at nunc commodo placerat. Praesent blandit. Nam nulla. Integer pede justo, lacinia eget, tincidunt eget, tempus vel, pede. Morbi porttitor lorem id ligula.',
  'in_stock': 37,
  'tags': ['Alcohol', 'Wine'],
  'category': ['food']}}

## Get Mapping

In [ ]:
es.